# Come NON diventare ricchi con Python

Questa sessione è pensata per dare qualche spunto pratico a chi ha la necessità di manipolare i dati usando Python e SQL Server.

Il notebook che state guardando è solo la prima parte, giusto per prendere confidenza con Python e con le librerie usate per la manipolazione dei dati (pandas e numpy) e quelle per il plotting dei dati (matplotlib e plotly).

Ci sono tonnellate di librerie per Python, alcune anche legate al mondo finanziario e questa sessione non ha assolutamente la pretesa di elencarle o dimostrarle tutte... A voi il divertimento ! :-)

  

Mi raccomando!!! Le demo che vedrete servono solamente a darvi qualche idea, **NON** usatele per giocare in borsa con soldi veri!!!

In [2]:
import datetime
import pandas as pd
import pandas_datareader as web
import numpy as np

ModuleNotFoundError: No module named 'pandas'

## Scaricare i dati finanziari da Yahoo Finance

Uno dei (pochi) siti da cui è possibile scaricare gratuitamente i valori degli scambi di borsa di titoli o ETF è Yahoo Finance.

Esempio: [https://finance.yahoo.com/quote/MSFT/history?p=MSFT](https://finance.yahoo.com/quote/MSFT/history?p=MSFT)

  

La libreria **pandas\_datareader** ha una funzione per scaricare i dati storici dal sito Yahoo Finance indicando il simbolo dell'azione/ETF ed i range di data di cui scaricare i dati.

In [2]:
#
# Demo - Scaricare i dati storici da Yahoo Finance
#
start_date = datetime.datetime(2020, 1, 1) 
end_date   = datetime.date.today()
symbol     = 'MSFT'

data = web.DataReader(symbol, 'yahoo', start_date, end_date)

#data.head(5) 

data.tail(5)

#print(data)

RemoteDataError: Unable to read URL: https://finance.yahoo.com/quote/MSFT/history?period1=1577847600&period2=1630807199&interval=1d&frequency=1d&filter=history
Response Text:
b'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n  }\n

## Che cosa fare con i dati scaricati ?

Una volta scaricati i dati storici di un azione o ETF si possono calcolare alcuni indicatori che sono alla base delle più comuni strategie di trading.

Quelli che utilizzeremo in questi esempi sono:

\- Media mobile esponenziale (Exponential Mobile Average, EMA)

\- Media mobile semplice (Simple Moving Average, SMA)

\- Convergenza e divergenza di medie mobili (Moving average convergence divergence, MACD) [Wikipedia](https://it.wikipedia.org/wiki/MACD)

\- Bande di Bollinger

In [ ]:
#
# Demo - Calcolo gli indicatori statistici
#

data.reset_index(inplace=True,drop=False)

# Medie mobili a 12 e 26 giorni
data['12d_EMA'] = data.Close.ewm(span=12, adjust=False).mean()
data['26d_EMA'] = data.Close.ewm(span=26, adjust=False).mean()

# Bollinger bands
data['20d_SMA'] = data['Close'].rolling(window=20).mean()
data['20d_STD'] = data['Close'].rolling(window=20).std()
data['Bollinger_H'] = data['20d_SMA'] + (data['20d_STD'] * 2)
data['Bollinger_L'] = data['20d_SMA'] - (data['20d_STD'] * 2)

# MACD
data['macd'] = data['12d_EMA'] - data['26d_EMA'] 
data['macdsignal'] = data.macd.ewm(span=9, adjust=False).mean()
data['trading_signal'] = np.where(data['macd'] >= data['macdsignal'], 1, -1)

data.tail(5)

## Plotting dei dati

Non è proprio facilissimo leggere la tabella appena vista, giusto?  
Allora per semplificarci la vita ed avere un "colpo d'occhio" più efficace nel capire se è il momento di acquistare o di vendere un bel grafico è quello che ci vuole!

La libreria **matplotlib** ci fornisce la possibilità di creare grafici in modo molto semplice e veloce...

In [ ]:
#
# Demo - Plot degli indicatori usando la libreria matplotlib
#
import matplotlib.pyplot as plt

data_to_plot = data #data.iloc[-30:]

plt.style.use('seaborn-muted')

fig, ax = plt.subplots(1, 2, figsize=(28, 10))

ax[0].set(title = 'Indicators - ' + symbol, ylabel = 'Price in Euros', xlabel = 'Date')
data_to_plot[['Date','Close','12d_EMA','26d_EMA','Bollinger_H','Bollinger_L']].plot(ax=ax[0], x='Date')

ax[1].set_title('MACD - ' + symbol)
data_to_plot[['Date','macd','macdsignal']].plot(ax=ax[1], x='Date')

plt.legend()

#filename = basePath + ticker + '_1.png'
#plt.savefig(filename)

plt.show()

Oppure posso utilizzare la libreria **Plotly**, più interattiva e con alcune tipologie di grafici già pronte all'uso per i dati finanziari (es. i grafici candlestick)

In [ ]:
#
# Demo - Plot degli indcatori usando la libreria Plotly
#
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Creo il grafico per gli ultimi 90 giorni
data_to_plot = data.iloc[-90:]

# Imposto la data come indice del dataframe
data_to_plot.set_index('Date', inplace=True)

# Creo tre grafici, in verticale, di altezza variabile
fig = make_subplots(rows=3,
                    cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02,
                    row_width=[0.1, 0.1, 0.8]
                    )

# Primo grafico: candlestick con i dati di minimo, massimo, apertura e chiusura degli scambi
fig.add_trace(
    go.Candlestick(
        x=data_to_plot.index,
        low=data_to_plot['Low'],
        high=data_to_plot['High'],
        close=data_to_plot['Close'],
        open=data_to_plot['Open'],
        increasing_line_color='green',
        decreasing_line_color='red'
    ),
    row=1,
    col=1
)

# Secondo grafico: volume degli scambi del titolo
fig.add_trace(go.Bar(x=data_to_plot.index, y=data_to_plot['Volume']), row=2, col=1)

# Terzo grafico: segnale di trading calcolato sull'indicatore MACD visto in precedenza
fig.add_trace(go.Bar(x=data_to_plot.index, y=data_to_plot['trading_signal']), row=3, col=1)

# Impostazioni grafico
fig.layout.xaxis1.type = 'category'
fig.layout.xaxis2.type = 'category'
fig.layout.xaxis3.type = 'category'

today = datetime.datetime.today().strftime("%d-%m-%Y")
title = "Interactive CandleStick & Volume Chart {symbol} - {date}".format(symbol=symbol, date=today)

fig.update_layout(title=title,
                yaxis1_title='Close (€)',
                yaxis2_title='Volume (M)',
                xaxis2_title='Time',
                xaxis1_rangeslider_visible=False,
                xaxis2_rangeslider_visible=False)

# Visualizzo il grafico
fig.show()

# Se voglio, posso salvare il grafico (statico) in formato png/jpeg/tiff
#filename = "./images/%s.png" % symbol
#fig.write_image(filename, width=1024, height=800, scale=2)


## Bene, e ora?

Fin qui tutto chiaro? Bene.

Ora viene la parte più divertente, memorizzare i dati su SQL Server ed attingere dal database per leggere i dati, calcolare indicatori e magari generare notifiche quando cambia il trading signal di un azione o ETF.

  

Let's go!